In [27]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
import cv2

In [28]:
def load_large_image_cv2(image_path):
    """Use OpenCV to load very large images"""
    try:
        print("Loading with OpenCV...")
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if img is None:
            raise ValueError("Could not load image")
            
        # Convert BGR to RGB if needed
        if len(img.shape) == 3 and img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
        print(f"Image shape: {img.shape}")
        return img
        
    except Exception as e:
        print(f"OpenCV loading failed: {e}")
        return None

In [41]:
# --- Load Nebula image ---
distorted_img = load_large_image_cv2('data/heic2007a_tampered_RB_right_Ggray.png')

# --- Step 0: Fix Red/Blue inversion (swap channels if inverted) ---
# The file seems to have R and B swapped compared to the reference
distorted_img = distorted_img[:,:, [2,1,0]]  # swap R <-> B

# --- Work on copy ---
restored = distorted_img.copy().astype(np.int16)

# --- Step 1: Circular shift Red & Blue channels back by 200 ---
restored[:,:,0] = (restored[:,:,0] - 200) % 256  # Red
restored[:,:,2] = (restored[:,:,2] - 200) % 256  # Blue

# --- Step 2: Rebuild Green channel (grayscale + small R+B mix) ---
green_gray = distorted_img[:,:,1].astype(np.float32)  # grayscale green
green_mix = ((restored[:,:,0] + restored[:,:,2]) / 2).astype(np.float32)

# Weighted blend: keep structure from gray, slight hint of color
green_reconstructed = 0.85 * green_gray + 0.15 * green_mix
green_reconstructed = np.clip(green_reconstructed, 0, 255).astype(np.uint8)

# Assign reconstructed green
restored[:,:,1] = green_reconstructed

# --- Final conversion ---
restored = restored.astype(np.uint8)

# Save result
cv2.imwrite('output/restored_heic2007a.jpg', cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))
print("heic2007a restored! Red & Blue circularly shifted back, Green reconstructed.")

Loading with OpenCV...
Image shape: (11710, 17043, 4)
heic2007a restored! Red & Blue circularly shifted back, Green reconstructed.
